In [1]:
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.1 MB/s eta 0:00:00


In [3]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 14.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.2
    Uninstalling scipy-1.16.2:
      Successfully uninstalled scipy-1.16.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatib

In [1]:
import pandas as pd
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import spacy
from datetime import datetime
import re
import string
import unicodedata
import random
from gensim import corpora, models

### Sentiment analysis
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

### Load Reddit Data

In [5]:
# import pandas as pd
url = 'https://raw.githubusercontent.com/EunCheolChoi0123/COMM557Tutorial/refs/heads/main/Tutorial%203%20Reddit%20Scraper%20and%20Network/datascience-new-comment.csv'
df  = pd.read_csv(url)
df = df[~df.body.isna()]
docs = df.body.to_list()

### Preprocessing text

In [22]:
import nltk
from nltk import ToktokTokenizer
import spacy
import re, unicodedata


nlp = spacy.load('en_core_web_sm')

### Run this the first time
nltk.download('stopwords')
nltk.download('punkt')

tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')


def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

def strip_links(text: str) -> str:
    SIMPLE_LINK_RE = re.compile(r'https?://\S+|www\.\S+')
    return SIMPLE_LINK_RE.sub(' ', text)

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

def noun_only(text, include_proper=True, lowercase=True, unique=False):
    if not isinstance(text, str) or not text.strip():
        return ""
    doc = nlp(text)
    keep_pos = {"NOUN", "PROPN"} if include_proper else {"NOUN"}
    out = [
        (tok.text.lower() if lowercase else tok.text)
        for tok in doc
        if tok.pos_ in keep_pos and tok.is_alpha and not tok.is_stop
    ]
    if unique:
        out = list(dict.fromkeys(out))
    return " ".join(out)


def process_posts(post):
    post = remove_stopwords(post)
    post = strip_links(post)
    post = remove_special_characters(post)
    post = lemmatize_text(post)
    post = noun_only(post)

    return post

df.loc[:, 'body'] = df.body.apply(process_posts)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
def process_posts(post):
    post = strip_links(post)
    post = remove_special_characters(post)
    return post

In [24]:
processed_docs = [process_posts(doc) for doc in docs]

In [25]:
processed_docs[30]

'Thanks'

### Tokenizing and Corpus Creation

In [26]:
#Create corupus of all words
words_corpus = []
for elem in processed_docs:
    # remove stop words
    elem = remove_stopwords(elem)
    # lemmatize text
    elem = lemmatize_text(elem)
    words_corpus.append(elem.lower().split())
print(len(words_corpus))

dictionary = corpora.Dictionary(words_corpus)
print(len(dictionary))

109
1278


In [27]:
dictionary.num_docs, dictionary.num_pos

(109, 3563)

In [28]:
# get token-id mapping
dictionary.token2id

{'ai': 0,
 'depend': 1,
 'give': 2,
 'hood': 3,
 'look': 4,
 'one': 5,
 'pivot': 6,
 'point': 7,
 'solid': 8,
 'sort': 9,
 'starting': 10,
 'want': 11,
 'add': 12,
 'basic': 13,
 'build': 14,
 'c': 15,
 'datum': 16,
 'etc': 17,
 'first': 18,
 'focus': 19,
 'get': 20,
 'i': 21,
 'know': 22,
 'little': 23,
 'm': 24,
 'mca': 25,
 'ml': 26,
 'portfolio': 27,
 'problemsolve': 28,
 'program': 29,
 'project': 30,
 'python': 31,
 'right': 32,
 'science': 33,
 'september': 34,
 'skill': 35,
 'small': 36,
 'sql': 37,
 'start': 38,
 'stat': 39,
 'student': 40,
 'suggestion': 41,
 'try': 42,
 'worth': 43,
 'collection': 44,
 'detailed': 45,
 'good': 46,
 'oh': 47,
 'really': 48,
 's': 49,
 'super': 50,
 'thank': 51,
 'that': 52,
 'yeah': 53,
 'dedicated': 54,
 'thread': 55,
 'use': 56,
 '2': 57,
 '30': 58,
 'actual': 59,
 'actually': 60,
 'although': 61,
 'analyticsds': 62,
 'analyze': 63,
 'anymore': 64,
 'anyone': 65,
 'back': 66,
 'bother': 67,
 'break': 68,
 'bring': 69,
 'budget': 70,
 'capit

In [29]:
dictionary.most_common(10)

[('not', 45),
 ('i', 44),
 ('job', 42),
 ('get', 37),
 ('use', 34),
 ('would', 34),
 ('do', 30),
 ('science', 29),
 ('like', 29),
 ('you', 26)]

In [30]:
# get bag-of-words repesentation of documents: list of (token_id, token_count) tuples
dictionary.doc2bow(words_corpus[0])

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1)]

## Topic Modeling
# https://radimrehurek.com/gensim/models/ldamodel.html

In [31]:
?dictionary.filter_extremes

In [32]:
dictionary = corpora.Dictionary(words_corpus)
dictionary.filter_extremes(no_below=10, no_above=0.2, keep_n=1000)

corpus_bow = [dictionary.doc2bow(text) for text in words_corpus]

# Term Frequency - Inverse Document Frequency

from gensim import corpora, models

tfidf = models.TfidfModel(corpus_bow)
corpus_tfidf = tfidf[corpus_bow]

In [33]:
corpus_tfidf.corpus

[[(0, 1), (1, 1), (2, 1), (3, 1)],
 [(4, 2), (5, 1), (6, 3), (7, 1), (8, 1), (9, 1), (10, 2)],
 [(11, 1), (12, 1), (13, 1), (14, 1)],
 [],
 [(14, 1)],
 [(0, 2),
  (3, 1),
  (5, 1),
  (6, 1),
  (15, 1),
  (16, 1),
  (17, 5),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 3),
  (23, 1),
  (24, 1),
  (25, 1)],
 [(12, 1), (17, 2), (26, 1)],
 [],
 [(7, 1),
  (17, 1),
  (21, 1),
  (23, 1),
  (24, 2),
  (27, 1),
  (28, 2),
  (29, 1),
  (30, 1),
  (31, 1)],
 [(4, 1), (7, 1), (16, 1), (18, 2), (22, 1), (30, 1), (32, 1)],
 [(0, 1), (18, 2), (20, 1), (23, 1), (33, 1)],
 [(12, 1), (13, 1), (14, 1), (18, 1), (32, 1), (34, 1)],
 [(6, 1), (21, 1), (24, 1), (26, 1), (30, 2), (35, 1)],
 [(21, 1)],
 [(35, 1)],
 [(26, 1), (36, 1), (37, 1)],
 [(7, 1), (12, 1), (32, 1)],
 [(11, 1), (18, 1), (20, 1), (23, 1), (26, 2), (38, 1), (39, 1)],
 [],
 [],
 [(2, 2),
  (13, 2),
  (18, 2),
  (21, 2),
  (22, 2),
  (24, 2),
  (26, 1),
  (30, 1),
  (32, 1),
  (33, 1),
  (36, 1),
  (38, 1),
  (40, 2),
  (41, 3)],
 [(2,

In [34]:
# Only 5 topics this time for simplicity
num_topics = 5
lda_model_tfidf = models.LdaMulticore(corpus_tfidf, num_topics=num_topics, id2word=dictionary, passes=4, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1, num_words=20):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.088*"work" + 0.066*"job" + 0.048*"take" + 0.045*"like" + 0.044*"much" + 0.037*"feel" + 0.035*"build" + 0.035*"something" + 0.034*"current" + 0.034*"role" + 0.033*"d" + 0.031*"company" + 0.030*"new" + 0.030*"time" + 0.029*"one" + 0.027*"think" + 0.024*"go" + 0.023*"say" + 0.023*"want" + 0.022*"datum"
Topic: 1 Word: 0.069*"re" + 0.067*"you" + 0.064*"look" + 0.060*"good" + 0.052*"really" + 0.043*"job" + 0.042*"year" + 0.042*"well" + 0.033*"also" + 0.030*"s" + 0.028*"experience" + 0.028*"one" + 0.022*"lot" + 0.022*"want" + 0.022*"m" + 0.021*"new" + 0.020*"current" + 0.019*"make" + 0.019*"right" + 0.019*"thing"
Topic: 2 Word: 0.110*"datum" + 0.083*"give" + 0.070*"want" + 0.054*"do" + 0.053*"m" + 0.053*"role" + 0.043*"look" + 0.040*"might" + 0.036*"learn" + 0.031*"new" + 0.029*"keep" + 0.029*"start" + 0.027*"thing" + 0.025*"one" + 0.022*"project" + 0.021*"data" + 0.018*"current" + 0.018*"pay" + 0.017*"time" + 0.016*"something"
Topic: 3 Word: 0.094*"do" + 0.080*"make" + 0.061

## Visualizing Topic Modeling

In [35]:
!pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 21.8 MB/s eta 0:00:00


In [36]:
import pyLDAvis
import pyLDAvis.gensim

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [37]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model_tfidf, corpus_tfidf, dictionary)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.040720 -0.076715       1        1  28.215009
4     -0.137286 -0.001285       2        1  21.988981
0      0.012526  0.023374       3        1  21.720300
2      0.044389  0.103120       4        1  15.325188
3      0.039651 -0.048495       5        1  12.750522, topic_info=       Term      Freq     Total Category  logprob  loglift
14    thank  7.000000  7.000000  Default  30.0000  30.0000
5     datum  6.000000  6.000000  Default  29.0000  29.0000
0      give  3.000000  3.000000  Default  28.0000  28.0000
17       do  6.000000  6.000000  Default  27.0000  27.0000
26     make  4.000000  4.000000  Default  26.0000  26.0000
..      ...       ...       ...      ...      ...      ...
16     data  0.245099  3.425370   Topic5  -4.6573  -0.5777
42    learn  0.203421  2.921240   Topic5  -4.8437  -0.6049
15  company  0.203322  2.992335   Topic5  -4.8442  -0.6294
10    start  0.228763  3.465325   Topic5  -4.7263  -0.6583
25     work  0.308893  5.280679   Topic5  -4.4260  -0.7792

[187 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
38        1  0.461997   also
38        5  0.461997   also
4         2  0.266411  build
4         3  0.532823  build
4         5  0.266411  build
...     ...       ...    ...
37        1  0.482048   year
37        2  0.241024   year
37        5  0.241024   year
41        1  0.758228    you
41        3  0.189557    you

[132 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 1, 3, 4])

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

# On Your Own
1. Scrape 1,000 submissions from a subreddit using Tutorial 2 script. You don't have to scrape comments, user data, etc. for this session.
2. Conduct LDA Topic Modeling. How many topics make the best sense in your opinion?
3. (Optional) Name each topics.